In [12]:
import csv
import xml.etree.ElementTree as ET
import json
import csv
import pyodbc
from geopy.geocoders import Nominatim
import requests
from datetime import datetime

In [13]:
# Load participant age, status, and type dictionaries
with open('dict_partecipant_age.json') as f1:
    dict_partecipant_age = json.load(f1)

with open('dict_partecipant_status.json') as f2:
    dict_partecipant_status = json.load(f2)

with open('dict_partecipant_type.json') as f3:
    dict_partecipant_type = json.load(f3)

In [14]:
# Function to compute additional date-related data
def compute_date_data(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    date = date_obj.date()
    day = date_obj.day
    month = date_obj.month
    year = date_obj.year
    quarter = (date_obj.month - 1) // 3 + 1
    day_of_week = date_obj.strftime('%A')
    return date, day, month, year, quarter, day_of_week

In [15]:
# Function to parse dates.xml and create a mapping of date_fk to real date
def parse_dates_xml(xml_file):
    date_mapping = {}
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for row in root.findall('.//row'):
        date = row.find('date').text
        date_pk = int(row.find('date_pk').text)
        date_mapping[date_pk] = date

    return date_mapping


In [16]:
# Function to compute crime gravity using provided dictionaries
def compute_crime_gravity(x):
    gravity = dict_partecipant_age.get(x['participant_age_group'], 1) * \
              dict_partecipant_type.get(x['participant_type'], 1) * \
              dict_partecipant_status.get(x['participant_status'], 1)
    return gravity

In [17]:
def get_continent_by_country_code(country_code):
    if country_code:
        try:
            response = requests.get(f'https://restcountries.com/v3/alpha/{country_code.lower()}')
            data = response.json()
            continent = data[0]['region']
            return continent
        except Exception as e:
            print(f"Error fetching continent information: {e}")
    
    return None

In [18]:
def get_location_info(latitude, longitude):
    geolocator = Nominatim(user_agent="Lab_DSS_Group_ID_200")
    location = geolocator.reverse((latitude, longitude), language='en')

    if location is not None:
        address = location.address
        city = location.raw.get('address', {}).get('city') or location.raw.get('address', {}).get('town') or location.raw.get('address', {}).get('village') or location.raw.get('address', {}).get('county', None)

        state = location.raw.get('address', {}).get('state', None)
        #country = location.raw.get('address', {}).get('country', None) (Not calculated as the country is USA for all records, and the Continent is North America)

        #continent = get_continent_by_country_code(location.raw.get('address', {}).get('country_code', None)) 

        return {
            'city': city,
            'state': state,
            #'country': country,
            #'continent': continent
        }
    else:
        return None


In [19]:
#Write to my db
# Connection string
server = 'tcp:lds.di.unipi.it'
username = 'Group_ID_200'
password = '89VIG10K'
database = 'Group_ID_200_DB'
connectionString = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password

In [20]:
# Connect to the SQL Server database
conn = pyodbc.connect(connectionString)
cursor = conn.cursor()

In [21]:
geo_id_dict = {}
gun_id_dict = {}
partecipant_id_dict = {}
date_id_dict = {}
incident_id_dict = {}
custody_id_dict = {}

In [22]:
# Parse dates.xml to get date mapping
date_mapping = parse_dates_xml('dates.xml')

def bulk_insert_data(conn, cursor, table_name, columns, values_list):
    placeholders = ', '.join(['?' for _ in range(len(columns))])
    insert_query = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
    cursor.executemany(insert_query, values_list)
    conn.commit()

def get_surrogate_key(conn, cursor, table_name, key_dict):
    columns = ', '.join(key_dict.keys())
    placeholders = ', '.join(['?'] * len(key_dict))
    select_query = f'SELECT * FROM {table_name} WHERE ({columns}) = ({placeholders});'

    cursor.execute(select_query, list(key_dict.values()))
    result = cursor.fetchone()

    if result:
        return result[0]  # Assuming the surrogate key is in the first column
    else:
        insert_query = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
        cursor.execute(insert_query, list(key_dict.values()))
        conn.commit()

        cursor.execute(select_query, list(key_dict.values()))
        return cursor.fetchone()[0]

def split_and_integrate(csv_file):
    
    table_names = ['Custody', 'Geography', 'Gun', 'Date', 'Incident', 'Partecipant']

    for table_name in table_names:
        cursor.execute(f'DELETE FROM {table_name}')
        conn.commit()

    batch_size = 50
    batch_data = []

    with open(csv_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        next(reader)

        for row in reader:
            gun_stolen_bool = 1 if row["gun_stolen"] == 'Stolen' else 0
            gun_key_dict = {"is_stolen": gun_stolen_bool, "gun_type": row['gun_type']}
            gun_id = get_surrogate_key(conn, cursor, 'Gun', gun_key_dict)

            partecipant_key = {
                "age_group": row['participant_age_group'],
                "gender": row['participant_gender'],
                "type": row['participant_type'],
                "status": row['participant_status']
            }
            partecipant_id = get_surrogate_key(conn, cursor, 'Partecipant', partecipant_key)

            latitude, longitude = float(row['latitude']), float(row['longitude'])
            location_info = get_location_info(latitude, longitude)
            city = location_info["city"]
            state = location_info["state"]
            country = "United States"
            continent = "North America"
            geo_key = {
                "latitude": str(latitude), "longitude": str(longitude), "city": city, "state": state, "country": country,
                "continent": continent
            }
            geo_id = get_surrogate_key(conn, cursor, 'Geography', geo_key)

            date_id = int(row['date_fk'])
            date_value = date_mapping[date_id]
            date, day, month, year, quarter, day_of_week = compute_date_data(date_value)
            date_key = {
                "date_id": date_id, "the_date": date, "the_day": day, "the_month": month, "the_year": year,
                "quarter": quarter, "day_of_week": day_of_week
            }
            insert_data_with_ID(conn, cursor, date_id_dict, "Date", date_key)

            incident_id = int(row['incident_id'])
            incident_key = {"incident_id": incident_id}
            insert_data_with_ID(conn, cursor, incident_id_dict, "Incident", incident_key)

            custody_key = {
                "custody_id": row['custody_id'], "partecipant_id": partecipant_id, "gun_id": gun_id, "geo_id": geo_id,
                "date_id": date_id, "crime_gravity": compute_crime_gravity(row), "incident_id ": incident_id
            }

            batch_data.append(tuple(custody_key.values()))

            row_count += 1

            if row_count % batch_size == 0:
                bulk_insert_data(conn, cursor, 'Custody', columns, batch_data)
                batch_data = []

    bulk_insert_data(conn, cursor, 'Custody', columns, batch_data)

    cursor.close()
    conn.close()

split_and_integrate('Police.csv')


In [ ]:
# Function to insert data into the database with an automatically generated ID
def insert_data_with_ID(conn, cursor, id_dict, table_name, key_dict):
    key_tuple = tuple(key_dict.values())
    if key_tuple not in id_dict:
        id_dict[key_tuple] = next(iter(key_dict.values()))
        columns = ', '.join(key_dict.keys())
        placeholders = ', '.join(['?'] * len(key_dict))
        insert_query = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'

        cursor.execute(insert_query, list(key_dict.values()))

# Function to get or insert ID into the dictionary
def get_or_insert_id(conn, cursor, id_dict, table_name, key_dict):
    key_tuple = tuple(key_dict.values())
    if key_tuple in id_dict:
        return id_dict[key_tuple]
    else:
        last_inserted_id = insert_data_with_ID(conn, cursor, id_dict, table_name, key_dict)
        return last_inserted_id

# Modified split_and_integrate function for batch inserts
def split_and_integrate(csv_file):
    # Parse dates.xml to get date mapping
    date_mapping = parse_dates_xml('dates.xml')

    # List of table names in your database
    table_names = ['Custody', 'Geography', 'Gun', 'Date', 'Incident', 'Partecipant']

    # Clean the tables by deleting all records
    for table_name in table_names:
        cursor.execute(f'DELETE FROM {table_name}')
        conn.commit()

    # Read and process Police.csv
    with open(csv_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        next(reader)  # Skip the header row

        # Set the batch size
        batch_size = 10
        batch_data = []

        row_count = 0
        # ... (previous code remains unchanged)

        for row in reader:
            custody_id, participant_age_group, participant_gender, participant_status, participant_type, latitude, longitude, gun_stolen, gun_type, incident_id, date_fk = row

            gun_stolen_bool = 1 if row["gun_stolen"] == 'Stolen' else 0
            gun_key_dict = {"is_stolen": gun_stolen_bool, "gun_type": row['gun_type']}
            gun_id = get_or_insert_id(conn, cursor, gun_id_dict, 'Gun', gun_key_dict)

            participant_key = {
                "age_group": row['participant_age_group'],
                "gender": row['participant_gender'],
                "type": row['participant_type'],
                "status": row['participant_status']
            }
            participant_id = get_or_insert_id(conn, cursor, partecipant_id_dict, 'Partecipant', participant_key)

            latitude, longitude = float(row['latitude']), float(row['longitude'])
            location_info = get_location_info(latitude, longitude)
            city = location_info["city"]
            state = location_info["state"]
            country = "United States"
            continent = "North America"
            geo_key = {
                "latitude": str(latitude), "longitude": str(longitude), "city": city, "state": state, "country": country,
                "continent": continent
            }
            geo_id = get_or_insert_id(conn, cursor, geo_id_dict, 'Geography', geo_key)

            date_id = int(row['date_fk'])
            date_value = date_mapping[date_id]
            date, day, month, year, quarter, day_of_week = compute_date_data(date_value)
            date_key = {
                "date_id": date_id, "the_date": date, "the_day": day, "the_month": month, "the_year": year,
                "quarter": quarter, "day_of_week": day_of_week
            }
            insert_data_with_ID(conn, cursor, date_id_dict, "Date", date_key)

            incident_id = int(row['incident_id'])
            incident_key = {"incident_id": incident_id}
            insert_data_with_ID(conn, cursor, incident_id_dict, "Incident", incident_key)

            custody_id = row['custody_id']
            custody_key = {
                "custody_id": custody_id, "partecipant_id": participant_id, "gun_id": gun_id, "geo_id": geo_id,
                "date_id": date_id, "crime_gravity": compute_crime_gravity(row), "incident_id": incident_id
            }
            insert_data_with_ID(conn, cursor, custody_id_dict, 'Custody', custody_key)

            # Increment the row count
            row_count += 1

            # Commit in batches of 1000
            if row_count % batch_size == 0:
                print(row_count)
                conn.commit()

# Commit any remaining records
conn.commit()

# Call the function with the appropriate arguments
split_and_integrate('Police.csv')

# Close the database connection when done
cursor.close()
conn.close()



IntegrityError: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert the value NULL into column 'partecipant_id', table 'Group_ID_200_DB.Group_ID_200.Custody'; column does not allow nulls. INSERT fails. (515) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")

In [ ]:
# Function to insert data into the database in bulk
def bulk_insert_data(conn, cursor, table_name, columns, values_list):
    placeholders = ', '.join(['?' for _ in range(len(columns))])
    insert_query = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
    cursor.executemany(insert_query, values_list)
    conn.commit()

# Function to get or insert ID in bulk
def bulk_get_or_insert_id(conn, cursor, id_dict, table_name, key_dict_list):
    key_tuples = [tuple(key_dict.values()) for key_dict in key_dict_list]
    existing_keys = set(key_tuples) & set(id_dict.keys())
    new_keys = set(key_tuples) - existing_keys

    # Insert new keys
    new_key_dicts = [key_dict_list[key_tuples.index(key)] for key in new_keys]
    columns = ', '.join(new_key_dicts[0].keys())
    bulk_insert_data(conn, cursor, table_name, columns, [tuple(key_dict.values()) for key_dict in new_key_dicts])

    # Fetch IDs of all keys
    cursor.execute(f'SELECT * FROM {table_name} WHERE ({columns}) IN ({", ".join(["?" for _ in range(len(new_key_dicts[0]))])});', [tuple(key_dict.values()) for key_dict in key_dict_list])
    new_ids = cursor.fetchall()

    # Update id_dict with new IDs
    for i, key_tuple in enumerate(new_keys):
        id_dict[key_tuple] = new_ids[i][0]

    # Return IDs for all keys
    return [id_dict[key_tuple] for key_tuple in key_tuples]

# Function to split and integrate data
def split_and_integrate(csv_file):
    # Parse dates.xml to get date mapping
    date_mapping = parse_dates_xml('dates.xml')

    # List of table names in your database
    table_names = ['Custody', 'Geography', 'Gun', 'Date', 'Incident', 'Partecipant']

    # Clean the tables by deleting all records
    for table_name in table_names:
        cursor.execute(f'DELETE FROM {table_name}')
        conn.commit()

    # Read and process Police.csv
    with open(csv_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        next(reader)  # Skip the header row

        # Set the batch size
        batch_size = 50
        batch_data = []

        for row in reader:
            custody_id, participant_age_group, participant_gender, participant_status, participant_type, latitude, longitude, gun_stolen, gun_type, incident_id, date_fk = row

            gun_stolen_bool = 1 if row["gun_stolen"] == 'Stolen' else 0
            gun_key_dict = {"is_stolen": gun_stolen_bool, "gun_type": row['gun_type']}
            gun_id = get_or_insert_id(conn, cursor, gun_id_dict, 'Gun', [gun_key_dict])[0]

            partecipant_key = {
                "age_group": row['participant_age_group'],
                "gender": row['participant_gender'],
                "type": row['participant_type'],
                "status": row['participant_status']
            }
            partecipant_id = get_or_insert_id(conn, cursor, partecipant_id_dict, 'Partecipant', [partecipant_key])[0]

            # Get location information from latitude and longitude
            latitude, longitude = float(row['latitude']), float(row['longitude'])
            location_info = get_location_info(latitude, longitude)
            city = location_info["city"]
            state = location_info["state"]
            country = "United States"
            continent = "North America"
            geo_key = {
                "latitude": str(latitude), "longitude": str(longitude), "city": city, "state": state, "country": country,
                "continent": continent
            }
            geo_id = get_or_insert_id(conn, cursor, geo_id_dict, 'Geography', [geo_key])[0]

            # Normal ID, No Incremental Tables:
            date_id = int(row['date_fk'])
            date_value = date_mapping[date_id]
            date, day, month, year, quarter, day_of_week = compute_date_data(date_value)
            date_key = {
                "date_id": date_id, "the_date": date, "the_day": day, "the_month": month, "the_year": year,
                "quarter": quarter, "day_of_week": day_of_week
            }
            insert_data_with_ID(conn, cursor, date_id_dict, "Date", date_key)

            incident_id = int(row['incident_id'])
            incident_key = {"incident_id": incident_id}
            insert_data_with_ID(conn, cursor, incident_id_dict, "Incident", incident_key)

            custody_id = row['custody_id']
            custody_key = {
                "custody_id": custody_id, "partecipant_id": partecipant_id, "gun_id": gun_id, "geo_id": geo_id,
                "date_id": date_id, "crime_gravity": compute_crime_gravity(row), "incident_id ": incident_id
            }

            # Append data to the batch
            batch_data.append(tuple(custody_key.values()))

            # Increment the row count
            row_count += 1

            # Commit in batches
            if row_count % batch_size == 0:
                bulk_insert_data(conn, cursor, 'Custody', columns, batch_data)
                batch_data = []  # Reset batch data

    # Commit any remaining records
    bulk_insert_data(conn, cursor, 'Custody', columns, batch_data)

# ... (existing code)

# Call the function with the appropriate arguments
split_and_integrate('Police.csv')

# Close the database connection when done
cursor.close()
conn.close()


In [ ]:
# ... (your existing code)

# Initialize dictionaries to store existing IDs
geo_id_dict = {}
gun_id_dict = {}
partecipant_id_dict = {}
date_id_dict = {}
incident_id_dict = {}
custody_id_dict = {}

# Retrieve existing IDs from the 'Geography' table
cursor.execute("SELECT geo_id, latitude, longitude FROM Geography")
for row in cursor.fetchall():
    geo_id = row[0]
    latitude = row[1]
    longitude = row[2]
    geo_key = {"latitude": str(latitude), "longitude": str(longitude)}
    geo_id_dict[geo_id] = geo_key

# Retrieve existing IDs from the 'Gun' table
cursor.execute("SELECT gun_id, is_stolen, gun_type FROM Gun")
for row in cursor.fetchall():
    gun_id = row[0]
    is_stolen = row[1]
    gun_type = row[2]
    gun_key = {"is_stolen": is_stolen, "gun_type": gun_type}
    gun_id_dict[gun_id] = gun_key

# Retrieve existing IDs from the 'Partecipant' table
cursor.execute("SELECT partecipant_id, age_group, gender, type, status FROM Partecipant")
for row in cursor.fetchall():
    partecipant_id = row[0]
    age_group = row[1]
    gender = row[2]
    participant_type = row[3]
    status = row[4]
    partecipant_key = {"age_group": age_group, "gender": gender, "type": participant_type, "status": status}
    partecipant_id_dict[partecipant_id] = partecipant_key

# Retrieve existing IDs from the 'Date' table
cursor.execute("SELECT date_id, the_date, the_day, the_month, the_year, quarter, day_of_week FROM Date")
for row in cursor.fetchall():
    date_id = row[0]
    the_date = row[1]
    the_day = row[2]
    the_month = row[3]
    the_year = row[4]
    quarter = row[5]
    day_of_week = row[6]
    date_key = {
        "date_id": date_id, "the_date": the_date, "the_day": the_day, "the_month": the_month,
        "the_year": the_year, "quarter": quarter, "day_of_week": day_of_week
    }
    date_id_dict[date_id] = date_key

# Retrieve existing IDs from the 'Incident' table
cursor.execute("SELECT incident_id FROM Incident")
for row in cursor.fetchall():
    incident_id = row[0]
    incident_key = {"incident_id": incident_id}
    incident_id_dict[incident_id] = incident_key

# Retrieve existing IDs from the 'Custody' table
cursor.execute("SELECT custody_id, partecipant_id, gun_id, geo_id, date_id, crime_gravity, incident_id FROM Custody")
for row in cursor.fetchall():
    custody_id = row[0]
    partecipant_id = row[1]
    gun_id = row[2]
    geo_id = row[3]
    date_id = row[4]
    crime_gravity = row[5]
    incident_id = row[6]
    custody_key = {
        "custody_id": custody_id, "partecipant_id": partecipant_id, "gun_id": gun_id, "geo_id": geo_id,
        "date_id": date_id, "crime_gravity": crime_gravity, "incident_id": incident_id
    }
    custody_id_dict[custody_id] = custody_key

# ... (continue with your existing code)


In [ ]:
import csv
import itertools
import sqlite3  # Assuming you are using SQLite, you can replace this with your database library
from decimal import Decimal as Decimal


# Function to get existing IDs from the database and populate the dictionary
def get_existing_ids(conn, cursor, table_name, key_columns):
    existing_ids = {}
    cursor.execute(f'SELECT * FROM {table_name}')
    rows = cursor.fetchall()

    # Print the rows for debugging
    print("Rows:", rows)

    for row in rows:
        # Convert Decimal values to float
        key = tuple(float(row[key_column]) if isinstance(row[key_column], Decimal) else row[key_column] for key_column in key_columns)
        existing_ids[key] = row[-1]  # Assuming the ID is the last element in the tuple
    return existing_ids




# Function to insert data with ID into the database and update the dictionary
def insert_data_with_id(conn, cursor, id_dict, table_name, data):
    key = tuple(data[key_column] for key_column in id_dict[table_name]['key_columns'])
    if key in id_dict[table_name]['existing_ids']:
        data['id'] = id_dict[table_name]['existing_ids'][key]
    else:
        columns = ', '.join(data.keys())
        placeholders = ', '.join(['?' for _ in range(len(data))])
        cursor.execute(f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders})', list(data.values()))
        conn.commit()
        new_id = cursor.lastrowid
        data['id'] = new_id
        id_dict[table_name]['existing_ids'][key] = new_id
    return data['id']

# Your existing code...

# Initialize dictionaries to store existing IDs
geo_id_dict = {'existing_ids': get_existing_ids(conn, cursor, 'Geography', [1, 2])}  # Use column indices
gun_id_dict = {'existing_ids': get_existing_ids(conn, cursor, 'Gun', [1, 2])}  # Use column indices
partecipant_id_dict = {'existing_ids': get_existing_ids(conn, cursor, 'Partecipant', [1, 2, 3, 4])}  # Use column indices
date_id_dict = {'existing_ids': get_existing_ids(conn, cursor, 'Date', [0, 1, 2, 3, 4, 5])}  # Use column indices
incident_id_dict = {'existing_ids': get_existing_ids(conn, cursor, 'Incident', [0])}  # Use column indices
custody_id_dict = {'existing_ids': get_existing_ids(conn, cursor, 'Custody', [0, 1, 2, 3, 4, 5])}  # Use column indices


# Your existing code...
